In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('Rent_Contracts.csv')

In [ ]:
data.head()

In [ ]:
data

In [ ]:
categorical_columns = data.select_dtypes(include=['object']).columns
for column in categorical_columns:
    print(data[column].value_counts())
    print('\n')

In [ ]:
def update_usage(usage_type):
    if usage_type == 'Residential':
        return 'residential'
    else:
        return 'commercial'

In [ ]:
rent_data = data[data['property_usage_en'].isin(['Residential', 'Commercial', 'Industrial', 'Industrial / Commercial', 'Storage'])]

rent_data['property_usage'] = rent_data['property_usage_en'].apply(update_usage)

rent_data

In [ ]:
from scipy.stats import trim_mean

def trimmed_mean(series, proportion_to_cut=0.1):
    return trim_mean(series, proportion_to_cut)

In [ ]:
rent_data['contract_start_date'] = pd.to_datetime(rent_data['contract_start_date'], format='%d-%m-%Y')
# 
rent_data['year_month'] = rent_data['contract_start_date'].dt.to_period('M')
grouped_rent = rent_data.groupby(['year_month', 'property_usage'])['annual_amount'].apply(trimmed_mean, proportion_to_cut=0.1).reset_index()
grouped_rent.rename(columns={'annual_amount': 'average_annual_amount'}, inplace=True)
# 
grouped_rent

In [ ]:
transactions_data = pd.read_csv('Transactions.csv')

In [ ]:
transactions_data = transactions_data.loc[transactions_data['trans_group_id'] == 1]

transactions_data = transactions_data[transactions_data['property_usage_en'].isin(['Residential', 'Commercial', 'Industrial', 'Industrial / Commercial', 'Storage'])]

transactions_data['property_usage'] = transactions_data['property_usage_en'].apply(update_usage)


In [ ]:
transactions_data['instance_date'] = pd.to_datetime(transactions_data['instance_date'], format='%d-%m-%Y')

transactions_data['year_month'] = transactions_data['instance_date'].dt.to_period('M')
transactions_data['year'] = transactions_data['year_month'].dt.year
grouped_sale = transactions_data[transactions_data['year_month'].isin(grouped_rent['year_month'])].groupby(['year_month', 'property_usage'])['actual_worth'].apply(trimmed_mean, proportion_to_cut=0.1).reset_index()

In [ ]:
cap_value = transactions_data['meter_sale_price'].quantile(0.95)
sarimax_full_data = transactions_data[transactions_data['meter_sale_price'] < cap_value]

quarterly_data = sarimax_full_data.groupby(['year', 'property_usage'])['meter_sale_price'].median().reset_index()
# quarterly_data.set_index('instance_date', inplace=True)

# Расчет pct_change по кварталам
quarterly_data['price_change'] = quarterly_data.groupby('property_usage')['meter_sale_price'].pct_change()

# Заполнение NaN значений, которые возникли при расчете pct_change для первого значения в каждой группе
quarterly_data['price_change'].fillna(0, inplace=True)

In [ ]:
grouped_sale.rename(columns={'actual_worth': 'average_actual_worth'}, inplace=True)
grouped_sale['year'] = grouped_sale['year_month'].dt.year

grouped_sale = pd.merge(grouped_sale, quarterly_data, on=['year', 'property_usage'], how='left')
grouped_sale

In [ ]:
inflation_data = pd.read_csv('inflation.csv')

inflation_data['inflation'] = inflation_data['inflation'] / 100
inflation_data

In [ ]:
merged_data = pd.merge(grouped_rent, grouped_sale, on=['year_month', 'property_usage'])
merged_data['year'] = merged_data['year_month'].dt.year

merged_data = pd.merge(merged_data, inflation_data, on='year')

merged_data

In [ ]:
merged_data['year'] = merged_data['year'].astype(int)

merged_data['rate_of_return'] = (((merged_data['average_annual_amount'] * (1 + merged_data['inflation'])) / merged_data['average_actual_worth']) + merged_data['inflation'])

merged_data['full_rate_of_return'] = merged_data['rate_of_return'] + merged_data['price_change']

merged_data

In [ ]:
import matplotlib.pyplot as plt

plot_data = merged_data.loc[merged_data['year'] > 2009].groupby(['year', 'property_usage'])[['rate_of_return']].mean().reset_index()

# Построение графика
plt.figure(figsize=(12, 6))

# Уникальные типы недвижимости
property_usages = plot_data['property_usage'].unique()

# Построение графиков для каждого типа недвижимости
for property_usage in ['residential','commercial']:
    subset = plot_data[plot_data['property_usage'] == property_usage]
    plt.plot(subset['year'], subset['rate_of_return'] * 100, marker='o', label=property_usage)

plt.title('Rate of Rent Return for each year')
plt.xlabel('Year')
plt.ylabel('Rate of Return')
plt.legend()
plt.grid(True)
plt.show()

plot_data

In [ ]:
import matplotlib.pyplot as plt

plot_data = merged_data.loc[merged_data['year'] > 2009].groupby(['year', 'property_usage'])[['price_change']].mean().reset_index()

# Построение графика
plt.figure(figsize=(12, 6))

plot_data['year_month_str'] = plot_data['year'].astype(str)

# Уникальные типы недвижимости
property_usages = plot_data['property_usage'].unique()

# Построение графиков для каждого типа недвижимости
for property_usage in ['residential','commercial']:
    subset = plot_data[plot_data['property_usage'] == property_usage]
    plt.plot(subset['year_month_str'], subset['price_change'] * 100, marker='o', label=property_usage)

plt.title('Rate of Capital gain for each year')
plt.xlabel('Year')
plt.ylabel('Rate of Return')
plt.legend()
# plt.xticks(rotation=45)
plt.grid(True)
plt.show()

plot_data

In [ ]:
import matplotlib.pyplot as plt

plot_data = merged_data.loc[merged_data['year'] > 2009].groupby(['year', 'property_usage'])[['full_rate_of_return']].mean().reset_index()

# Построение графика
plt.figure(figsize=(12, 6))

# Уникальные типы недвижимости
property_usages = plot_data['property_usage'].unique()

# Построение графиков для каждого типа недвижимости
for property_usage in ['residential','commercial']:
    subset = plot_data[plot_data['property_usage'] == property_usage]
    plt.plot(subset['year'], subset['full_rate_of_return'] * 100, marker='o', label=property_usage)

plt.title('Rate of Return for each year')
plt.xlabel('Year')
plt.ylabel('Rate of Return')
plt.legend()
plt.grid(True)
plt.show()

plot_data

In [ ]:
rus_inflation = {
    "2024-04":7.84 / 100,
    "2024-03":7.72 / 100,
    "2024-02":7.69 / 100,
    "2024-01":7.44 / 100,
    "2023-12":7.42 / 100,
    "2023-11":7.48 / 100,
    "2023-10":6.69 / 100,
    "2023-09":6.00 / 100,
    "2023-08":5.15 / 100,
    "2023-07":4.30 / 100,
    "2023-06":3.25 / 100,
    "2023-05":2.51 / 100,
    "2023-04":2.31 / 100,
    "2023-03":3.51 / 100,
    "2023-02":10.99 / 100,
    "2023-01":11.77 / 100,
    "2022-12":11.94 / 100,
    "2022-11":11.98 / 100,
    "2022-10":12.63 / 100,
    "2022-09":13.68 / 100,
    "2022-08":14.30 / 100,
    "2022-07":15.10 / 100,
    "2022-06":15.90 / 100,
    "2022-05":17.10 / 100,
    "2022-04":17.83 / 100,
    "2022-03":16.69 / 100,
    "2022-02":9.15 / 100,
    "2022-01":8.73 / 100,
    "2021-12":8.39 / 100,
    "2021-11":8.40 / 100,
    "2021-10":8.13 / 100,
    "2021-09":7.40 / 100,
    "2021-08":6.68 / 100,
    "2021-07":6.50 / 100,
    "2021-06":6.50 / 100,
    "2021-05":6.00 / 100,
    "2021-04":5.50 / 100,
    "2021-03":5.80 / 100,
    "2021-02":5.70 / 100,
    "2021-01":5.2 / 100,
}

oae_inflation = {
    "2021-01": 0.15 / 100,
    "2021-02": 0.30 / 100,
    "2021-03": 0.40 / 100,
    "2021-04": 0.50 / 100,
    "2021-05": 0.60 / 100,
    "2021-06": 0.70 / 100,
    "2021-07": 0.80 / 100,
    "2021-08": 0.90 / 100,
    "2021-09": 1.00 / 100,
    "2021-10": 1.10 / 100,
    "2021-11": 1.20 / 100,
    "2021-12": 1.30 / 100,
    "2022-01": 1.40 / 100,
    "2022-02": 1.50 / 100,
    "2022-03": 1.60 / 100,
    "2022-04": 1.70 / 100,
    "2022-05": 1.80 / 100,
    "2022-06": 1.90 / 100,
    "2022-07": 2.00 / 100,
    "2022-08": 2.10 / 100,
    "2022-09": 2.20 / 100,
    "2022-10": 2.30 / 100,
    "2022-11": 2.40 / 100,
    "2022-12": 2.50 / 100,
    "2023-01": 2.4 / 100,
    "2023-02": 2.5 / 100,
    "2023-03": 2.6 / 100,
    "2023-04": 2.7 / 100,
    "2023-05": 2.8 / 100,
    "2023-06": 2.9 / 100,
    "2023-07": 3.0 / 100,
    "2023-08": 3.1 / 100,
    "2023-09": 3.2 / 100,
    "2023-10": 3.3 / 100,
    "2023-11": 3.4 / 100,
    "2023-12": 3.5 / 100,
    "2024-01": 3.6 / 100,
    "2024-02": 3.7 / 100,
    "2024-03": 3.8 / 100,
    "2024-04": 3.9 / 100,
    "2024-05": 3.9 / 100,
    "2024-06": 3.9 / 100,
    "2024-07": 3.9 / 100,
    "2024-08": 3.8 / 100,
    "2024-09": 3.7 / 100,
    "2024-10": 3.6 / 100,
    "2024-11": 3.6 / 100,
    "2024-12": 3.5 / 100,
    "2025-01": 3.3 / 100,
    "2025-02": 3.3 / 100,
    "2025-03": 3.3 / 100,
    "2025-04": 3.3 / 100,
    "2025-05": 3.3 / 100,
}

In [ ]:
residential_return = merged_data[(merged_data['property_usage'] == 'residential') & (merged_data['year'] > 2004)][['year', 'year_month', 'full_rate_of_return', 'price_change']]

residential_return.rename(columns={'full_rate_of_return': 'residential_return'}, inplace=True)
residential_return.rename(columns={'price_change': 'residential_capital_return'}, inplace=True)

residential_return

In [ ]:
commercial_return = merged_data[(merged_data['property_usage'] == 'commercial') & (merged_data['year'] > 2004)][['year', 'year_month', 'full_rate_of_return', 'price_change']]

commercial_return.rename(columns={'full_rate_of_return':'commercial_return'}, inplace=True)
commercial_return.rename(columns={'price_change':'commercial_capital_return'}, inplace=True)

commercial_return

In [ ]:
moex = pd.read_csv('MOEX Russia Historical Data.csv', thousands=',')
dividends = {
    2024: 17.35/12,
    2023: 4.84/12,
    2022: 0,
    2021: 9.45/12,
    2020: 7.93/12,
}
moex['Date'] = pd.to_datetime(moex['Date'], format='%m/%d/%Y')

moex['capital_return'] = moex['Price'].pct_change()
moex['year_month'] = moex['Date'].dt.to_period('M')
moex['year'] = moex['Date'].dt.year
moex['return'] = moex.apply(lambda row: row['capital_return'] + (dividends[row['year']] / row['Price']), axis=1)
moex['moex_real_return'] = moex.apply(lambda row: ((1 + row['return']) / (1 + rus_inflation[str(row['year_month'])]) - 1), axis=1)

moex = moex.drop(columns=['Open','High','Low','Vol.','Change %', 'Price'], axis=1)
moex = moex.dropna()

moex.rename(columns={'return': 'moex_return'}, inplace=True)

moex

In [ ]:
rtsi = pd.read_csv('RTSI Historical Data.csv', thousands=',')
rtsi['Date'] = pd.to_datetime(rtsi['Date'], format='%m/%d/%Y')

rtsi['return'] = rtsi['Price'].pct_change()
rtsi['year_month'] = rtsi['Date'].dt.to_period('M')
rtsi['year'] = rtsi['Date'].dt.year
rtsi['rtsi_real_return'] = rtsi.apply(lambda row: ((1 + row['return']) / (1 + rus_inflation[str(row['year_month'])]) - 1), axis=1)

rtsi = rtsi.drop(columns=['Open','High','Low','Vol.','Change %', 'Price'], axis=1)
rtsi = rtsi.dropna()

rtsi.rename(columns={'return': 'rtsi_return'}, inplace=True)

rtsi

In [ ]:
deposits = pd.read_csv('deposits.csv')

deposits['year_month'] = deposits['year_month'].apply(lambda x: pd.Period(x, freq='M'))
deposits['return'] = ((1 + deposits['annual_return']/100) ** (1/12)) - 1
deposits['deposits_real_return'] = deposits.apply(lambda row: ((1 + row['return']) / (1 + rus_inflation[str(row['year_month'])]) - 1), axis=1)

deposits.rename(columns={'return': 'deposits_return'}, inplace=True)

deposits

In [ ]:
rgbitr = pd.read_csv('rgbitr.csv',)
rgbitr['date'] = pd.to_datetime(rgbitr['date'], format='%d.%m.%Y')

rgbitr['return'] = rgbitr['price'].pct_change()
rgbitr['year_month'] = rgbitr['date'].dt.to_period('M')
rgbitr['year'] = rgbitr['date'].dt.year
rgbitr['rgbitr_real_return'] = rgbitr.apply(lambda row: ((1 + row['return']) / (1 + rus_inflation[str(row['year_month'])]) - 1), axis=1)

rgbitr = rgbitr.dropna()

rgbitr.rename(columns={'return': 'rgbitr_return'}, inplace=True)

rgbitr

In [ ]:
rucbtrns = pd.read_csv('rucbtrns.csv',)
rucbtrns['date'] = pd.to_datetime(rucbtrns['date'], format='%d.%m.%Y')

rucbtrns['return'] = rucbtrns['price'].pct_change()
rucbtrns['year_month'] = rucbtrns['date'].dt.to_period('M')
rucbtrns['year'] = rucbtrns['date'].dt.year
rucbtrns['rucbtrns_real_return'] = rucbtrns.apply(lambda row: ((1 + row['return']) / (1 + rus_inflation[str(row['year_month'])]) - 1), axis=1)
 
rucbtrns = rucbtrns.dropna()

rucbtrns.rename(columns={'return': 'rucbtrns_return'}, inplace=True)

rucbtrns

In [ ]:
merged_returns = commercial_return[commercial_return['year'] > 2021][['commercial_return', 'commercial_capital_return', 'year_month']].merge(residential_return[residential_return['year'] > 2021][['residential_return', 'residential_capital_return', 'year_month']], on='year_month')
merged_returns = merged_returns.merge(rtsi[rtsi['year'] > 2021][['rtsi_return', 'year_month']], on='year_month')
merged_returns = merged_returns.merge(moex[moex['year'] > 2021][['moex_return', 'year_month']], on='year_month')
merged_returns = merged_returns.merge(deposits[deposits['year_month'] > '2021'][['deposits_return', 'year_month']], on='year_month')
merged_returns = merged_returns.merge(rgbitr[rgbitr['year'] > 2021][['rgbitr_return', 'year_month']], on='year_month')
merged_returns = merged_returns.merge(rucbtrns[rucbtrns['year'] > 2021][['rucbtrns_return', 'year_month']], on='year_month')

merged_returns

In [ ]:
# Построение графика
plt.figure(figsize=(12, 6))

plot_data_merged = merged_returns

plot_data_merged['year_month_str'] = plot_data_merged['year_month'].astype(str)

plt.plot(plot_data_merged['year_month_str'], plot_data_merged['deposits_return'] * 100, label='Deposits')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['rgbitr_return'] * 100, label='RGBITR')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['rucbtrns_return'] * 100, label='RUCBTRNS')


# plt.title('Rate of Return')
plt.xlabel('Year')
plt.ylabel('Rate of Return')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
# Построение графика
plt.figure(figsize=(12, 6))

plot_data_merged = merged_returns

plot_data_merged['year_month_str'] = plot_data_merged['year_month'].astype(str)

plt.plot(plot_data_merged['year_month_str'], plot_data_merged['moex_return'] * 100, label='IMOEX')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['rtsi_return'] * 100, label='RTSI')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['residential_return'] * 100, label='Residential rate of return')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['commercial_return'] * 100, label='Commercial rate of return')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['deposits_return'] * 100, label='Deposits')
# plt.plot(plot_data_merged['year_month_str'], plot_data_merged['rgbitr_return'] * 100, label='RGBITR')
# plt.plot(plot_data_merged['year_month_str'], plot_data_merged['rucbtrns_return'] * 100, label='RUCBTRNS')


plt.title('Rate of Return')
plt.xlabel('Year')
plt.ylabel('Rate of Return')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
# Построение графика
plt.figure(figsize=(12, 6))

plot_data_merged = merged_returns

plot_data_merged['year_month_str'] = plot_data_merged['year_month'].astype(str)

plt.plot(plot_data_merged['year_month_str'], plot_data_merged['moex_return'] * 100, label='IMOEX')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['rtsi_return'] * 100, label='RTSI')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['residential_capital_return'] * 100, label='Residential capital return')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['commercial_capital_return'] * 100, label='Commercial capital return')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['deposits_return'] * 100, label='Deposits')
# plt.plot(plot_data_merged['year_month_str'], plot_data_merged['rgbitr_return'] * 100, label='RGBITR')
# plt.plot(plot_data_merged['year_month_str'], plot_data_merged['rucbtrns_return'] * 100, label='RUCBTRNS')


plt.title('Rate of Return')
plt.xlabel('Year')
plt.ylabel('Rate of Return')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
exchange_rate = pd.read_csv('AED_RUB Historical Data.csv')

exchange_rate['Date'] = pd.to_datetime(exchange_rate['Date'], format='%m/%d/%Y')

exchange_rate['exchange_rate_return'] = exchange_rate['Price'].pct_change()
exchange_rate['year_month'] = exchange_rate['Date'].dt.to_period('M')

exchange_rate = exchange_rate[['year_month', 'exchange_rate_return']]

In [ ]:
commercial_return_new = commercial_return.loc[commercial_return['year'] > 2021].copy()
residential_return_new = residential_return.loc[residential_return['year'] > 2021].copy()

commercial_return_new['commercial_real_return'] = commercial_return_new.apply(lambda row: ((1 + row['commercial_return']) / (1 + oae_inflation[str(row['year_month'])]) - 1), axis=1)
commercial_return_new['commercial_capital_real_return'] = commercial_return_new.apply(lambda row: ((1 + row['commercial_capital_return']) / (1 + oae_inflation[str(row['year_month'])]) - 1), axis=1)

residential_return_new['residential_real_return'] = residential_return_new.apply(lambda row: ((1 + row['residential_return']) / (1 + oae_inflation[str(row['year_month'])]) - 1), axis=1)
residential_return_new['residential_capital_real_return'] = residential_return_new.apply(lambda row: ((1 + row['residential_capital_return']) / (1 + oae_inflation[str(row['year_month'])]) - 1), axis=1)

commercial_return_new = commercial_return_new.merge(exchange_rate, on='year_month', how='left')


commercial_return_new['commercial_real_return_rub'] = ((1 + commercial_return_new['commercial_real_return']) * (1 + commercial_return_new['exchange_rate_return']) - 1)
commercial_return_new['commercial_capital_real_return_rub'] = ((1 + commercial_return_new['commercial_capital_real_return']) * (1 + commercial_return_new['exchange_rate_return']) - 1)

residential_return_new = residential_return_new.merge(exchange_rate, on='year_month', how='left')

residential_return_new['residential_real_return_rub'] = ((1 + residential_return_new['residential_real_return']) * (1 + residential_return_new['exchange_rate_return']) - 1)
residential_return_new['residential_capital_real_return_rub'] = ((1 + residential_return_new['residential_capital_real_return']) * (1 + residential_return_new['exchange_rate_return']) - 1)

commercial_return_new

In [ ]:
merged_returns = commercial_return_new[commercial_return_new['year'] > 2021][
    ['commercial_real_return_rub',
     'commercial_real_return', 
     'year_month', 
     'commercial_capital_real_return', 
     'commercial_capital_real_return_rub']
].merge(
    residential_return_new[residential_return_new['year'] > 2021][
        [
            'residential_real_return_rub', 
            'residential_real_return', 
            'year_month',
            'residential_capital_real_return',
            'residential_capital_real_return_rub']
    ], on='year_month')
merged_returns = merged_returns.merge(rtsi[rtsi['year'] > 2021][['rtsi_real_return', 'year_month']], on='year_month')
merged_returns = merged_returns.merge(moex[moex['year'] > 2021][['moex_real_return', 'year_month']], on='year_month')
merged_returns = merged_returns.merge(deposits[deposits['year_month'] > '2021'][['deposits_real_return', 'year_month']], on='year_month')
merged_returns = merged_returns.merge(rgbitr[rgbitr['year_month'] > '2021'][['rgbitr_real_return', 'year_month']], on='year_month')
merged_returns = merged_returns.merge(rucbtrns[rucbtrns['year_month'] > '2021'][['rucbtrns_real_return', 'year_month']], on='year_month')

merged_returns

In [ ]:
# Построение графика
plt.figure(figsize=(12, 6))

plot_data_merged = merged_returns

plot_data_merged['year_month_str'] = plot_data_merged['year_month'].astype(str)

plt.plot(plot_data_merged['year_month_str'], plot_data_merged['moex_real_return'] * 100, label='IMOEX')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['rtsi_real_return'] * 100, label='RTSI')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['residential_real_return'] * 100, label='Residential rate of return')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['commercial_real_return'] * 100, label='Commercial rate of return')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['deposits_real_return'] * 100, label='Deposits')


plt.title('Investment Real Returns')
plt.xlabel('Year')
plt.ylabel('Rate of Return')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
# Построение графика
plt.figure(figsize=(12, 6))

plot_data_merged = merged_returns

plot_data_merged['year_month_str'] = plot_data_merged['year_month'].astype(str)

plt.plot(plot_data_merged['year_month_str'], plot_data_merged['moex_real_return'] * 100, label='IMOEX')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['rtsi_real_return'] * 100, label='RTSI')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['residential_capital_real_return'] * 100, label='Residential capital return')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['commercial_capital_real_return'] * 100, label='Commercial capital return')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['deposits_real_return'] * 100, label='Deposits')

plt.title('Investment Real Returns')
plt.xlabel('Year')
plt.ylabel('Rate of Return')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
# Построение графика
plt.figure(figsize=(12, 6))

plot_data_merged = merged_returns

plot_data_merged['year_month_str'] = plot_data_merged['year_month'].astype(str)

# plt.plot(plot_data_merged['year_month_str'], plot_data_merged['moex_real_return'] * 100, label='IMOEX')
# plt.plot(plot_data_merged['year_month_str'], plot_data_merged['rtsi_real_return'] * 100, label='RTSI')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['residential_real_return_rub'] * 100, label='Residential rate of return')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['commercial_real_return_rub'] * 100, label='Commercial rate of return')
# plt.plot(plot_data_merged['year_month_str'], plot_data_merged['deposits_real_return'] * 100, label='Deposits')

plt.title('Real Rate of Returns')
plt.xlabel('Year')
plt.ylabel('Rate of Return in RUB')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
# Построение графика
plt.figure(figsize=(12, 6))

plot_data_merged = merged_returns

plot_data_merged['year_month_str'] = plot_data_merged['year_month'].astype(str)

plt.plot(plot_data_merged['year_month_str'], plot_data_merged['moex_real_return'] * 100, label='IMOEX')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['rtsi_real_return'] * 100, label='RTSI')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['residential_capital_real_return_rub'] * 100, label='Residential capital return')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['commercial_capital_real_return_rub'] * 100, label='Commercial capital return')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['deposits_real_return'] * 100, label='Deposits')

plt.title('Real Rate of Returns')
plt.xlabel('Year')
plt.ylabel('Rate of Return in RUB')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
capital = 1000

for rate in moex[moex['year'] > 2021].sort_values(by='Date')['moex_real_return']:
    capital = capital * (1 + ((1+rate) ** (1/12) - 1))

capital

In [ ]:
capital = 1000

for rate in rtsi[rtsi['year'] > 2021].sort_values(by='Date')['rtsi_real_return']:
    capital = capital * (1 + ((1+rate) ** (1/12) - 1))

capital

In [ ]:
capital = 1000

for rate in residential_return_new.sort_values(by='year_month')['residential_real_return_rub']:
    capital = capital * (1 + ((1+rate) ** (1/12) - 1))

capital

In [ ]:
capital = 1000

for rate in commercial_return_new.sort_values(by='year_month')['commercial_real_return_rub']:
    capital = capital * (1 + ((1+rate) ** (1/12) - 1))

capital

In [ ]:
capital = 1000

for rate in deposits[deposits['year_month'] > '2021'].sort_values(by='year_month')['deposits_real_return']:
    capital = capital * (1 + ((1+rate) ** (1/12) - 1))

capital

In [ ]:
capital = 1000

for rate in residential_return_new.sort_values(by='year_month')['residential_capital_real_return_rub']:
    capital = capital * (1 + ((1+rate) ** (1/12) - 1))

capital

In [ ]:
capital = 1000

for rate in commercial_return_new.sort_values(by='year_month')['commercial_capital_real_return_rub']:
    capital = capital * (1 + ((1+rate) ** (1/12) - 1))

capital

In [ ]:
predicted_transactions = pd.read_csv('predicted_12m.csv')

predicted_transactions['date'] = pd.to_datetime(predicted_transactions['date'])

predicted_transactions['year_month'] = predicted_transactions['date'].dt.to_period('M')
grouped_sale_predicted = predicted_transactions[predicted_transactions['year_month'].isin(grouped_rent['year_month'])]

# 
grouped_sale_predicted

In [ ]:
grouped_rent_for_prediction = rent_data.groupby(['year_month', 'property_usage']).apply(lambda x: pd.Series({
        'annual_amount': trimmed_mean(x['annual_amount'], proportion_to_cut=0.1),
        'actual_area': trimmed_mean(x['actual_area'], proportion_to_cut=0.1)
    })).reset_index()
grouped_rent_for_prediction.rename(columns={'annual_amount': 'average_annual_amount'}, inplace=True)
# 
grouped_rent_for_prediction

In [ ]:
merged_data_predicted = pd.merge(grouped_rent_for_prediction, grouped_sale_predicted, on=['year_month', 'property_usage'])
merged_data_predicted['year'] = merged_data_predicted['date'].dt.year

merged_data_predicted = pd.merge(merged_data_predicted, inflation_data, on='year')

merged_data_predicted = merged_data_predicted[merged_data_predicted['year_month'] > '2024-04']

In [ ]:
merged_data_predicted['predicted_sale_price'] = merged_data_predicted['predicted_mean'] * merged_data_predicted['actual_area']

merged_data_predicted.loc[(merged_data_predicted['year_month'] == '2024-06') & (merged_data_predicted['property_usage'] == 'commercial'), 'average_annual_amount'] = 203775.939024

merged_data_predicted

In [ ]:
merged_data_predicted['year'] = merged_data_predicted['year'].astype(int)

merged_data_predicted['rate_of_return'] = (((merged_data_predicted['average_annual_amount'] * (1 + merged_data_predicted['inflation'])) / merged_data_predicted['predicted_sale_price']) + merged_data_predicted['inflation'])

merged_data_predicted

In [ ]:
import matplotlib.pyplot as plt

plot_data = merged_data_predicted

# # Построение графика
plt.figure(figsize=(12, 6))

# # Уникальные типы недвижимости
property_usages = plot_data['property_usage'].unique()
# 
# # Построение графиков для каждого типа недвижимости
plot_data['year_month'] = plot_data['year_month'].astype(str)

# Построение графика
for property_usage in ['residential', 'commercial']:
    subset = plot_data[plot_data['property_usage'] == property_usage]
    plt.plot(subset['year_month'], subset['rate_of_return'] * 100, marker='o', label=property_usage)
# 
plt.title('Rate of Return for each year')
plt.xlabel('Year')
plt.ylabel('Rate of Return')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
merged_data_predicted['year_month'] = merged_data_predicted['year_month'].astype(str)
exchange_rate_new = exchange_rate
exchange_rate_new['year_month'] = exchange_rate_new['year_month'].astype(str)
merged_data_predicted = merged_data_predicted.merge(exchange_rate, on='year_month', how='left')

In [ ]:
merged_data_predicted['real_return'] = merged_data_predicted.apply(lambda row: ((1 + row['rate_of_return']) / (1 + oae_inflation[str(row['year_month'])]) - 1), axis=1)

merged_data_predicted['real_return_rub'] = ((1 + merged_data_predicted['real_return']) * (1 + merged_data_predicted['exchange_rate_return']) - 1)

merged_data_predicted

In [ ]:
# Построение графика
plt.figure(figsize=(12, 6))

plot_data_merged = merged_data_predicted

plot_data_merged['year_month_str'] = plot_data_merged['year_month'].astype(str)

res = plot_data_merged.loc[plot_data_merged['property_usage'] == 'residential']
com = plot_data_merged.loc[plot_data_merged['property_usage'] == 'commercial']

plt.plot(res['year_month_str'], res['real_return'] * 100, label='Residential real estate')
plt.plot(com['year_month_str'], com['real_return'] * 100, label='Commercial real estate')

plt.title('Real Rate of Returns')
plt.xlabel('Year')
plt.ylabel('Rate of Return')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
# Построение графика
plt.figure(figsize=(12, 6))

plot_data_merged = merged_data_predicted

plot_data_merged['year_month_str'] = plot_data_merged['year_month'].astype(str)

res = plot_data_merged.loc[plot_data_merged['property_usage'] == 'residential']
com = plot_data_merged.loc[plot_data_merged['property_usage'] == 'commercial']

plt.plot(res['year_month_str'], res['real_return_rub'] * 100, label='Residential real estate')
plt.plot(com['year_month_str'], com['real_return_rub'] * 100, label='Commercial real estate')

plt.title('Real Rate of Returns')
plt.xlabel('Year')
plt.ylabel('Rate of Return in RUB')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
reit = pd.read_csv('REIT Historical Data (1).csv', thousands=',')

reit['Date'] = pd.to_datetime(reit['Date'], format='%m/%d/%Y')

reit['capital_return'] = reit['Price'].pct_change()
reit['year_month'] = reit['Date'].dt.to_period('M')
reit['year'] = reit['Date'].dt.year
reit['return'] = reit['capital_return']
reit['reit_real_return'] = reit.apply(lambda row: ((1 + row['return']) / (1 + oae_inflation[str(row['year_month'])]) - 1), axis=1)

reit = reit.drop(columns=['Open','High','Low','Vol.','Change %', 'Price'], axis=1)
reit = reit.dropna()

reit.rename(columns={'return': 'reit_return'}, inplace=True)

reit

In [ ]:
merged_returns = commercial_return_new[commercial_return_new['year'] > 2021][
    ['commercial_real_return_rub',
     'commercial_real_return', 
     'year_month', 
     'commercial_capital_real_return', 
     'commercial_capital_real_return_rub']
].merge(
    residential_return_new[residential_return_new['year'] > 2021][
        [
            'residential_real_return_rub', 
            'residential_real_return', 
            'year_month',
            'residential_capital_real_return',
            'residential_capital_real_return_rub']
    ], on='year_month')
merged_returns = merged_returns.merge(reit[reit['year'] > 2021][['reit_real_return', 'year_month']], on='year_month')

In [ ]:
# Построение графика
plt.figure(figsize=(12, 6))

plot_data_merged = merged_returns

plot_data_merged['year_month_str'] = plot_data_merged['year_month'].astype(str)

plt.plot(plot_data_merged['year_month_str'], plot_data_merged['reit_real_return'] * 100, label='Emirates REIT CEIC')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['residential_real_return_rub'] * 100, label='Residential rate of return')
plt.plot(plot_data_merged['year_month_str'], plot_data_merged['commercial_real_return_rub'] * 100, label='Commercial rate of return')

plt.title('Real Rate of Returns')
plt.xlabel('Year')
plt.ylabel('Rate of Return')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.show()